### train.ipynb
This notebook is for training on Google Colab.

In [ ]:
# necessary imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm
from dataload import MyDataset
# import all the models
from lgg_model import *

In [ ]:
option = input("请选择是否储存单词模型（y/n）：")
if option == 'y' or option == 'Y':
    save_model = True
else:
    save_model = False

In [ ]:
# generate dataset and dataloader
BS = 32
input_word_count = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_path = input("请输入数据集名称（无需添加后缀）：")
data_path = "datasets/" + data_path + ".txt"
full_dataset = MyDataset(data_path, input_word_count, save_word_model=save_model) # set the save option to True for the first time
vocabulary_length = full_dataset.vocabulary_length

train_in_all = 0.8 # train_in_all is the proportion of the dataset that will be used for training

train_size = int(train_in_all * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
train_dataloader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BS, shuffle=True)

In [ ]:
vocabulary_length

In [ ]:
from datetime import datetime as dt

now = str(dt.now())
time_path = now[:10] + "_" + now[11:13] + "_" + now[14:16] + "_" + now[17:19]
print(time_path)
tb_path = "hole/" + time_path
tb_train_path = tb_path + "/train"
tb_val_path = tb_path + "/val"
tb_train_path, tb_val_path

In [ ]:
from torch.utils.tensorboard import SummaryWriter 
writer_train = SummaryWriter(tb_train_path)
writer_val = SummaryWriter(tb_val_path)

In [ ]:
# some components in the training process
LR = 0.001 # the learning rate of 0.001 is still too large, maybe needs lr_decay or batch_norm
num_epoches = 50
net = vanilla_GRU(vocabulary_length, 50, 100, 2).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR) 
criterion = nn.CrossEntropyLoss()

# 2022/2/27 add a lr decay controller
lr_decay_rate = 0.98
ctrl = torch.optim.lr_scheduler.ExponentialLR(optimizer, lr_decay_rate)

In [ ]:
# print the hyperparameters
print("The hyperparameters are as follows:")
print("The dataset is:", data_path)
print("The learning rate is:", LR)
print("The number of epochs is:", num_epoches)
print("The batch size is:", BS)
print("The input word count is:", input_word_count)
print("The vocabulary length is:", vocabulary_length)
print("The ratio of train/val is:", int(train_in_all/(1-train_in_all)), ":", 1)
print("The network is:", net)
print("The device is:", device)
print("The optimizer is:", optimizer)
print("The criterion is:", criterion)
print("The lr decay schedule is:", ctrl, ", and the lr decay rate is:", lr_decay_rate)

In [ ]:
# start training!
for epoch in tqdm(range(num_epoches)):
    # train
    net.train()
    train_loss = 0
    for i, data in enumerate(train_dataloader):
        data = data.to(device)
        data = data.to(torch.long)
        label = data[:,1:]
        out = net(data)[:,:-1,:]
        out = torch.transpose(out, 2, 1)

        optimizer.zero_grad()
        loss = criterion(out, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_avg_loss = train_loss / len(train_dataloader)
    writer_train.add_scalar('Loss/Epoch', train_avg_loss, epoch+1) # epoch+1 because epoch starts from 0
    writer_train.flush()
    ctrl.step() # lr decay
    
    # validation
    net.eval()
    val_loss = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            data = data.to(device)
            data = data.to(torch.long)
            label = data[:,1:]
            out = net(data)[:,:-1,:]
            out = torch.transpose(out, 2, 1)
            loss = criterion(out, label)
            val_loss += loss.item()
    
    val_avg_loss = val_loss / len(val_dataloader)
    writer_val.add_scalar('Loss/Epoch', val_avg_loss, epoch+1) # epoch+1 because epoch starts from 0
    writer_val.flush()
    
print("Finish training!")

In [ ]:
# if you want to save your language model...
# model_name = input("请输入语言模型的名称：")
model_name = "hole-merge"
model_name = model_name + '_' + time_path
torch.save(net, "lgg_model_paths/" + model_name)

In [ ]:
print("Now the learning rate is:", optimizer.param_groups[0]['lr'])